<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Victor", "transactions": [{"transaction-id": 1, "amount": 1610}, {"transaction-id": 1277, "amount": 2452}, {"transaction-id": 2046, "amount": 1729}, {"transaction-id": 2164, "amount": 1850}, {"transaction-id": 2368, "amount": 2163}, {"transaction-id": 3705, "amount": 2749}, {"transaction-id": 4579, "amount": 2252}, {"transaction-id": 4878, "amount": 1885}, {"transaction-id": 5791, "amount": 2001}, {"transaction-id": 6362, "amount": 1873}, {"transaction-id": 6960, "amount": 1837}, {"transaction-id": 8057, "amount": 1928}, {"transaction-id": 9697, "amount": 2560}]}\n',)

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Victor", "transactions": [{"transaction-id": 1, "amount": 1610}, {"transaction-id": 1277, "amount": 2452}, {"transaction-id": 2046, "amount": 1729}, {"transaction-id": 2164, "amount": 1850}, {"transaction-id": 2368, "amount": 2163}, {"transaction-id": 3705, "amount": 2749}, {"transaction-id": 4579, "amount": 2252}, {"transaction-id": 4878, "amount": 1885}, {"transaction-id": 5791, "amount": 2001}, {"transaction-id": 6362, "amount": 1873}, {"transaction-id": 6960, "amount": 1837}, {"transaction-id": 8057, "amount": 1928}, {"transaction-id": 9697, "amount": 2560}]}\n',
 '{"id": 1, "name": "Ray", "transactions": [{"transaction-id": 114, "amount": 1409}, {"transaction-id": 204, "amount": 1385}, {"transaction-id": 223, "amount": 1386}, {"transaction-id": 592, "amount": 1355}, {"transaction-id": 600, "amount": 1442}, {"transaction-id": 729, "amount": 1304}, {"transaction-id": 969, "amount": 1400}, {"transaction-id": 2010, "amount": 1438}, {"transaction-id": 2032, "amount

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Victor',
  'transactions': [{'transaction-id': 1, 'amount': 1610},
   {'transaction-id': 1277, 'amount': 2452},
   {'transaction-id': 2046, 'amount': 1729},
   {'transaction-id': 2164, 'amount': 1850},
   {'transaction-id': 2368, 'amount': 2163},
   {'transaction-id': 3705, 'amount': 2749},
   {'transaction-id': 4579, 'amount': 2252},
   {'transaction-id': 4878, 'amount': 1885},
   {'transaction-id': 5791, 'amount': 2001},
   {'transaction-id': 6362, 'amount': 1873},
   {'transaction-id': 6960, 'amount': 1837},
   {'transaction-id': 8057, 'amount': 1928},
   {'transaction-id': 9697, 'amount': 2560}]},
 {'id': 1,
  'name': 'Ray',
  'transactions': [{'transaction-id': 114, 'amount': 1409},
   {'transaction-id': 204, 'amount': 1385},
   {'transaction-id': 223, 'amount': 1386},
   {'transaction-id': 592, 'amount': 1355},
   {'transaction-id': 600, 'amount': 1442},
   {'transaction-id': 729, 'amount': 1304},
   {'transaction-id': 969, 'amount': 1400},
   {'transaction-

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 81,
  'name': 'Alice',
  'transactions': [{'transaction-id': 279, 'amount': 3838},
   {'transaction-id': 345, 'amount': 3818},
   {'transaction-id': 481, 'amount': 4023},
   {'transaction-id': 517, 'amount': 3766},
   {'transaction-id': 574, 'amount': 3983},
   {'transaction-id': 1076, 'amount': 3853},
   {'transaction-id': 1235, 'amount': 4109},
   {'transaction-id': 1588, 'amount': 4392},
   {'transaction-id': 1616, 'amount': 4033},
   {'transaction-id': 1651, 'amount': 3967},
   {'transaction-id': 1670, 'amount': 4122},
   {'transaction-id': 1967, 'amount': 3709},
   {'transaction-id': 2058, 'amount': 4053},
   {'transaction-id': 2552, 'amount': 4047},
   {'transaction-id': 2743, 'amount': 3619},
   {'transaction-id': 2800, 'amount': 3689},
   {'transaction-id': 2968, 'amount': 4171},
   {'transaction-id': 3003, 'amount': 4009},
   {'transaction-id': 3193, 'amount': 3892},
   {'transaction-id': 3304, 'amount': 4131},
   {'transaction-id': 3328, 'amount': 3811},
   {'transact

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 63},
 {'name': 'Alice', 'count': 56},
 {'name': 'Alice', 'count': 5},
 {'name': 'Alice', 'count': 26},
 {'name': 'Alice', 'count': 8})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(63, 56, 5, 26, 8)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

33.512727272727275

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 279, 'amount': 3838},
  {'transaction-id': 345, 'amount': 3818},
  {'transaction-id': 481, 'amount': 4023},
  {'transaction-id': 517, 'amount': 3766},
  {'transaction-id': 574, 'amount': 3983},
  {'transaction-id': 1076, 'amount': 3853},
  {'transaction-id': 1235, 'amount': 4109},
  {'transaction-id': 1588, 'amount': 4392},
  {'transaction-id': 1616, 'amount': 4033},
  {'transaction-id': 1651, 'amount': 3967},
  {'transaction-id': 1670, 'amount': 4122},
  {'transaction-id': 1967, 'amount': 3709},
  {'transaction-id': 2058, 'amount': 4053},
  {'transaction-id': 2552, 'amount': 4047},
  {'transaction-id': 2743, 'amount': 3619},
  {'transaction-id': 2800, 'amount': 3689},
  {'transaction-id': 2968, 'amount': 4171},
  {'transaction-id': 3003, 'amount': 4009},
  {'transaction-id': 3193, 'amount': 3892},
  {'transaction-id': 3304, 'amount': 4131},
  {'transaction-id': 3328, 'amount': 3811},
  {'transaction-id': 3343, 'amount': 3784},
  {'transaction-id': 3378, 'amount': 

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 279, 'amount': 3838},
 {'transaction-id': 345, 'amount': 3818},
 {'transaction-id': 481, 'amount': 4023})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(3838, 3818, 4023)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1106.6663953993057

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 130), ('Alice', 134), ('Alice', 143), ('Alice', 143), ('Bob', 144), ('Bob', 144), ('Bob', 156), ('Bob', 156), ('Charlie', 120), ('Charlie', 120), ('Charlie', 130), ('Charlie', 130), ('Dan', 81), ('Dan', 91), ('Dan', 170), ('Edith', 120), ('Edith', 120), ('Edith', 130), ('Edith', 130), ('Frank', 130), ('Frank', 130), ('Frank', 240), ('George', 132), ('George', 132), ('George', 143), ('George', 143), ('Hannah', 107), ('Hannah', 108), ('Hannah', 116), ('Hannah', 117), ('Ingrid', 132), ('Ingrid', 132), ('Ingrid', 143), ('Ingrid', 143), ('Jerry', 107), ('Jerry', 108), ('Jerry', 115), ('Jerry', 116), ('Kevin', 93), ('Kevin', 94), ('Kevin', 104), ('Kevin', 104), ('Laura', 126), ('Laura', 128), ('Laura', 139), ('Laura', 141), ('Michael', 107), ('Michael', 108), ('Michael', 116), ('Michael', 117), ('Norbert', 144), ('Norbert', 144), ('Norbert', 153), ('Norbert', 156), ('Oliver', 96), ('Oliver', 96), ('Oliver', 103), ('Oliver', 104), ('Patricia', 144), ('Patricia', 144), ('Patricia', 

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 550), ('Bob', 600), ('Charlie', 500), ('Dan', 342), ('Edith', 500), ('Frank', 500), ('George', 550), ('Hannah', 448), ('Ingrid', 550), ('Jerry', 446), ('Kevin', 395), ('Laura', 534), ('Michael', 448), ('Norbert', 597), ('Oliver', 399), ('Patricia', 600), ('Quinn', 550), ('Ray', 450), ('Sarah', 699), ('Tim', 198), ('Ursula', 419), ('Victor', 249), ('Wendy', 300), ('Xavier', 842), ('Yvonne', 300), ('Zelda', 391)]
CPU times: user 170 ms, sys: 12.2 ms, total: 182 ms
Wall time: 622 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Victor,"[{'transaction-id': 1, 'amount': 1610}, {'tran..."
1,1,Ray,"[{'transaction-id': 114, 'amount': 1409}, {'tr..."
2,2,Frank,"[{'transaction-id': 43, 'amount': 637}, {'tran..."
3,3,Edith,"[{'transaction-id': 384, 'amount': 69}, {'tran..."
4,4,Norbert,"[{'transaction-id': 22, 'amount': 5311}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 229 ms, sys: 12.5 ms, total: 242 ms
Wall time: 1.7 s


name
Alice      550
Bob        600
Charlie    500
Dan        342
Edith      500
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Victor', 'amount': 1610, 'transaction-id': 1},
 {'id': 0, 'name': 'Victor', 'amount': 2452, 'transaction-id': 1277},
 {'id': 0, 'name': 'Victor', 'amount': 1729, 'transaction-id': 2046})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Victor,1610,1
1,0,Victor,2452,1277
2,0,Victor,1729,2046
3,0,Victor,1850,2164
4,0,Victor,2163,2368


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 221 ms, sys: 22.3 ms, total: 244 ms
Wall time: 1.5 s


name
Alice       18432
Bob         36421
Charlie     24271
Dan         10030
Edith       21576
Frank       22166
George      20704
Hannah      16232
Ingrid      33400
Jerry        7961
Kevin       11691
Laura       23128
Michael     13093
Norbert     28762
Oliver      13681
Patricia    20139
Quinn       16461
Ray         12405
Sarah       31500
Tim          7081
Ursula       8076
Victor       7914
Wendy       28400
Xavier      47468
Yvonne      11784
Zelda        7224
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()